In [1]:
##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
mainPath = r'\\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\SummariesWithProcessAusGov'
outPath = r'P:\projects\RC11_ResultsSets'
summariesOutFolder = 'WilkinsonSedimentBudgets'
baseFolderEnd = 'BASE_RC11'
regContributorFileName = 'RegToOutlet.csv'
outputCSVEnd = 'SedimentBudgets.csv'
baselineScenarioName = 'Baseline'
reportCardString = 'RC2020'

#pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'


##altRegLinker = 'Catchmt'
#standardRegName = 'RepReg'
##altRegColName = 'AUSGMCAS'
#origLinker = 'ModelElement'

#origLoadToStream = 'LoadToStream (kg)'
#origLoadToExport = 'LoadToRegExport (kg)'
annLoadToStream = "AnnLoadToStreamKG"
annLoadToExport = "AnnLoadToExportKG"

#regionIDs = ['MW']
regionIDs = ['BM','BU','CY','FI','MW','WT']
Constituents = ['Sediment - Fine']
Processes = ['Gully', 'Hillslope surface soil', 'Streambank']
#FUs = ['Sugarcane', 'Grazing Open', 'Grazing Forested', 'Bananas']

scenarioToFolderDict = {baselineScenarioName:baseFolderEnd}


In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

In [3]:
if not os.path.exists(outPath + '\\' + summariesOutFolder):
    os.makedirs(outPath + '\\' + summariesOutFolder)
    print("Made dir: " + outPath + '\\' + summariesOutFolder)

Made dir: P:\projects\RC11_ResultsSets\WilkinsonSedimentBudgets


In [4]:
def getPathInfo(regionIDString, scenarioString):
    fileIn = mainPath + '\\' + regionIDString + '_' + reportCardString + '_' + baselineScenarioName + '_' + regContributorFileName
    fileOut = outPath + '\\' + summariesOutFolder + '\\' + regionIDString + '_' + reportCardString + '_' + scenarioString + '_' + outputCSVEnd
    return fileIn, fileOut


In [5]:
def produceSummaryFromRegContrib(regionIDString, scenarioName):
    
    theFileIn, theFileOut = getPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    rawcontrib = pd.read_csv(theFileIn)
    
    filteredConstit = rawcontrib.loc[rawcontrib['Constituent'].isin(Constituents)]
    processFiltered = filteredConstit.loc[filteredConstit['Process'].isin(Processes)]
    
    procForArea = Processes[0]#Probably Gully
    areaFiltered = processFiltered.loc[processFiltered['Process'] == procForArea]
    areaTotals = pd.DataFrame(areaFiltered.groupby(['Rep_Region']).agg({'AreaHA':'sum'})).reset_index()
    
    regNonProcessRSDR = pd.DataFrame(processFiltered.groupby(['Rep_Region']).agg({annLoadToStream:'sum', annLoadToExport:'sum'})).reset_index()
    regNonProcessRSDR['RegRSDR'] = regNonProcessRSDR[annLoadToExport] / regNonProcessRSDR[annLoadToStream]
    #where 1 is the axis number (0 for rows and 1 for columns.)
    regNonProcessRSDR = regNonProcessRSDR.drop(annLoadToStream, 1)
    regNonProcessRSDR.rename(columns={annLoadToExport:'RegTotalAnnLoadToExportKG'}, inplace=True)
    #regNonProcessRSDR = regNonProcessRSDR.drop(annLoadToExport, 1)  
    
    regProcessTotals = pd.DataFrame(processFiltered.groupby(['Rep_Region','Process']).agg({annLoadToStream:'sum'})).reset_index()
    regProcessTotals = pd.merge(regProcessTotals, regNonProcessRSDR, how='left', left_on=['Rep_Region'], right_on = ['Rep_Region'])
    regProcessTotals['AdjAnnLoadToExportKG'] = regProcessTotals[annLoadToStream] * regProcessTotals['RegRSDR']
    regProcessTotals['PercExport'] = regProcessTotals['AdjAnnLoadToExportKG'] / regProcessTotals['RegTotalAnnLoadToExportKG'] * 100
    regProcessTotals = pd.merge(regProcessTotals, areaTotals, how='left', left_on=['Rep_Region'], right_on = ['Rep_Region'])
    

    regProcessTotals.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [6]:
#List out the processes

for theReg in regionIDs:
    
    for scenario in scenarioToFolderDict.keys():
        
        print("Doing: " + theReg + " " + scenario)
        produceSummaryFromRegContrib(theReg, scenario)



print("Finished first summary")

Doing: BM Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\BM_RC2020_Baseline_RegToOutlet.csv Scenario: Baseline
Saved P:\projects\RC11_ResultsSets\WilkinsonSedimentBudgets\BM_RC2020_Baseline_SedimentBudgets.csv
Doing: BU Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\BU_RC2020_Baseline_RegToOutlet.csv Scenario: Baseline


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


Saved P:\projects\RC11_ResultsSets\WilkinsonSedimentBudgets\BU_RC2020_Baseline_SedimentBudgets.csv
Doing: CY Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\CY_RC2020_Baseline_RegToOutlet.csv Scenario: Baseline
Saved P:\projects\RC11_ResultsSets\WilkinsonSedimentBudgets\CY_RC2020_Baseline_SedimentBudgets.csv
Doing: FI Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\FI_RC2020_Baseline_RegToOutlet.csv Scenario: Baseline
Saved P:\projects\RC11_ResultsSets\WilkinsonSedimentBudgets\FI_RC2020_Baseline_SedimentBudgets.csv
Doing: MW Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_RC2020_ResultsSets_PointOfTruth\SummariesWithProcessAusGov\MW_RC2020_Baseline_RegToOutlet.csv Scenario: Baseline
Saved P:\projects\RC11_ResultsSets\WilkinsonSedimentBudgets\MW_RC2020_Baseline_SedimentBudgets.csv
Doing: WT Baseline
Processing \\athenasmb\pdreef\RC11_RC2020\RC11_R